In [ ]:
import os, json, glob, datetime, gc, re, time

from urllib import parse
import sqlalchemy as sa
import pyodbc
import psycopg2

In [ ]:
creds_file = r"D:\personal\creds\psql\db_creds.json"
with open(creds_file) as f:
    creds = json.load(f)

uid = creds["uid"]
pw = creds["password"]
host = "localhost"
port = 5432
db = "spotify"

engine = sa.create_engine(f"postgresql+psycopg2://{uid}:{pw}@{host}:{port}/{db}", pool_pre_ping=True, echo=True)

In [ ]:
conn = engine.connect()

In [ ]:
conn.execute("drop schema audio, temp cascade")

In [ ]:
stmt = """
CREATE SCHEMA IF NOT EXISTS audio;
CREATE TABLE audio.albums (
id                            text PRIMARY KEY,
name                          text,
label                         text,
album_type                    text,
available_markets             text[],
genres                        text[],
release_date                  date,
release_date_precision        text,
total_tracks                  integer,
popularity                    integer,
type                          text,
uri                           text,
track_ids                     text[],
artist_ids                    text[],
upload_dt                     timestamp
);

CREATE TABLE audio.artists (
id                            text PRIMARY KEY,
name                          text,
genres                        text[],
type                          text,
popularity                    integer,
followers                     integer,
uri                           text,
upload_dt                     timestamp
);

CREATE TABLE audio.track_info (
id                            text PRIMARY KEY,
name                          text,
type                          text,
available_markets             text,
is_local                      text,
disc_number                   integer,
track_number                  integer,
explicit                      text,
duration_ms                   integer,
popularity                    integer,
uri                           text,
album_id                      text,
artist_ids                    text[],
upload_dt                     timestamp
);

CREATE TABLE audio.track_features (
id                            text PRIMARY KEY,
danceability                  float,
energy                        float,
key                           integer,
loudness                      float,
mode                          integer,
speechiness                   float,
acousticness                  float,
instrumentalness              float,
liveness                      float,
valence                       float,
tempo                         float,
uri                           text,
duration_ms                   integer,
time_signature                integer
);

CREATE TABLE audio.track_analysis (
id                            text PRIMARY KEY,         
num_samples                   integer,
duration                      float,
offset_seconds                integer,
window_seconds                integer,
analysis_sample_rate          integer,
analysis_channels             integer,
end_of_fade_in                float,
start_of_fade_out             float,
loudness                      float,
tempo                         float,
tempo_confidence              float,
time_signature                integer,
time_signature_confidence     float,
key                           integer,
key_confidence                float,
mode                          integer,
mode_confidence               float,
upload_dt                     timestamp
);

CREATE TABLE audio.track_bars (
id                            text,
start                         float,
duration                      float,
confidence                    float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE audio.track_beats (
id                            text,
start                         float,
duration                      float,
confidence                    float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE audio.track_sections (
id                            text,
start                         float,
duration                      float,
confidence                    float,
loudness                      float,
tempo                         float,
tempo_confidence              float,
key                           integer,
key_confidence                float,
mode                          integer,
mode_confidence               float,
time_signature                integer,
time_signature_confidence     float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE audio.track_segments (
id                            text,
start                         float,
duration                      float,
confidence                    float,
loudness_start                float,
loudness_max_time             float,
loudness_max                  float,
loudness_end                  float,
pitches                       float[],
timbre                        float[],
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE audio.track_tatums (
id                            text, 
start                         float,
duration                      float,
confidence                    float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE audio.track_genres (
id                            text PRIMARY KEY,
genres                        text[]
);
"""

In [ ]:
[conn.execute(s) for s in stmt.split(";") if s != "\n"]

In [ ]:
stmt = """
CREATE SCHEMA IF NOT EXISTS temp;
CREATE TABLE temp.albums (
id                            text PRIMARY KEY,
name                          text,
label                         text,
album_type                    text,
available_markets             text[],
genres                        text[],
release_date                  date,
release_date_precision        text,
total_tracks                  integer,
popularity                    integer,
type                          text,
uri                           text,
track_ids                     text[],
artist_ids                    text[],
upload_dt                     timestamp
);

CREATE TABLE temp.artists (
id                            text PRIMARY KEY,
name                          text,
genres                        text[],
type                          text,
popularity                    integer,
followers                     integer,
uri                           text,
upload_dt                     timestamp
);

CREATE TABLE temp.track_info (
id                            text PRIMARY KEY,
name                          text,
type                          text,
available_markets             text,
is_local                      text,
disc_number                   integer,
track_number                  integer,
explicit                      text,
duration_ms                   integer,
popularity                    integer,
uri                           text,
album_id                      text,
artist_ids                    text[],
upload_dt                     timestamp
);

CREATE TABLE temp.track_features (
id                            text PRIMARY KEY,
danceability                  float,
energy                        float,
key                           integer,
loudness                      float,
mode                          integer,
speechiness                   float,
acousticness                  float,
instrumentalness              float,
liveness                      float,
valence                       float,
tempo                         float,
uri                           text,
duration_ms                   integer,
time_signature                integer
);

CREATE TABLE temp.track_analysis (
id                            text PRIMARY KEY,         
num_samples                   integer,
duration                      float,
offset_seconds                integer,
window_seconds                integer,
analysis_sample_rate          integer,
analysis_channels             integer,
end_of_fade_in                float,
start_of_fade_out             float,
loudness                      float,
tempo                         float,
tempo_confidence              float,
time_signature                integer,
time_signature_confidence     float,
key                           integer,
key_confidence                float,
mode                          integer,
mode_confidence               float,
upload_dt                     timestamp
);

CREATE TABLE temp.track_bars (
id                            text,
start                         float,
duration                      float,
confidence                    float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE temp.track_beats (
id                            text,
start                         float,
duration                      float,
confidence                    float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE temp.track_sections (
id                            text,
start                         float,
duration                      float,
confidence                    float,
loudness                      float,
tempo                         float,
tempo_confidence              float,
key                           integer,
key_confidence                float,
mode                          integer,
mode_confidence               float,
time_signature                integer,
time_signature_confidence     float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE temp.track_segments (
id                            text,
start                         float,
duration                      float,
confidence                    float,
loudness_start                float,
loudness_max_time             float,
loudness_max                  float,
loudness_end                  float,
pitches                       float[],
timbre                        float[],
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);

CREATE TABLE temp.track_tatums (
id                            text, 
start                         float,
duration                      float,
confidence                    float,
upload_dt                     timestamp,
PRIMARY KEY (id, start)
);
"""

[conn.execute(s) for s in stmt.split(";") if s != "\n"]